##### Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

#################
# My notebook issues
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
sys.path.append('home/egupta/.local/bin')

##############

import os
import numpy as np
import scipy.stats as ss
import scipy
import networkx as nx
import csv 
import numpy as np
import seaborn as sns
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline


##qtensor imports
import qtensor
from qtensor import QAOA_energy
from qtensor import parameter_optimization as popt
import torch

2021-07-02 03:38:53,161- WARNING•	QuickBB solver is unavailable
2021-07-02 03:38:53,163- WARNING•	Tamaki solver is unavailable: No path /home/egupta/.local/lib/python3.7/site-packages/thirdparty/tamaki_treewidth


### Functions to generate graphs

In [2]:

def get_qaoa_params(graph, edge=None, gamma=[0.5], beta=[0.5], steps=50):
    loss_history, params_history = popt.qaoa_maxcut_torch(graph, gamma, beta,
                                                          ordering_algo='greedy',
                                                          Opt=torch.optim.RMSprop,
                                                          edge=edge,
                                                          opt_kwargs=dict(lr=0.002),
                                                          steps=steps,
                                                          pbar=False)
    best_ix = np.argmin(loss_history)
    return params_history[best_ix], loss_history[best_ix]

def get_edge_contribution(graph, edge, gamma, beta):
    sim = qtensor.QtreeSimulator()
    composer = qtensor.DefaultQAOAComposer(graph, gamma=gamma, beta=beta)
    composer.energy_expectation_lightcone(edge)
    return np.real(sim.simulate_batch(composer.circuit)[0])

In [3]:
def is_isomorphic_to_old(graph,g_list):
  """
  Checks if graph is iso.. to graphs already in csv file
  """
  for i in g_list:
    if nx.is_isomorphic(graph, i):
      return True
  return False

In [4]:
def generate_regular_graph(deg, nodes, g_list):
  """
  generates connected regular graph with specifed degree and nodes
  """
  G= nx.generators.random_graphs.random_regular_graph(deg, nodes)
  while (nx.is_connected(G) is False) or (is_isomorphic_to_old(G, g_list)):
    G = nx.generators.random_graphs.random_regular_graph(deg, nodes)
  return G

In [5]:
def opt_runs(G, seeds = 20, steps = 200):
  """
  Doing multiple optimization runs, each with random initialized param
  Returning results which inlclude the optimal param and corresponding energies
  """

  #first do 20 optimization runs 
  s = 0
  g_results = []
  max_index = 0
  max_energy = 0

  while s<seeds:
    print('seed #' + str(s))
    #initialize param
    gamma = [np.random.rand()]
    beta = [np.random.rand() ]
    initial_energy = QAOA_energy(G,gamma =  gamma, beta = beta)

    #optimization
    params, loss = get_qaoa_params(G, gamma=gamma, beta=beta,edge = None, steps=steps)
    #print('Finished get_qaoa_params')
    gamma_opt, beta_opt = params[0], params[1]
    opt_energy = QAOA_energy(G, gamma = gamma_opt, beta = beta_opt)

    result = np.asarray([float(gamma_opt), float(beta_opt), float(opt_energy), float(initial_energy)])
    g_results.append(result)

    # comparing with max
    if opt_energy >max_energy:
      max_index = s
      max_energy = opt_energy
    s+=1
 
  #adding last entru as the max one
  max_result = np.copy(g_results[max_index])
  
  return max_result

# Getting the remaining graphs

In [6]:
g_data = []
with open('16NodeRegularData_1_10_Jul1.csv', 'r', newline = '') as s:
  spamreader = csv.reader(s, quotechar='|')
  for row in spamreader:
    new_row = [float(i) for i in row]
    g_data.append(new_row)
first_rem_index = len(g_data)  #index of first graph whose data is not present

In [7]:
first_rem_index

27

In [14]:
resi = []
with open('16NodeRegularGraph_1_10_Jul1.csv', 'r', newline = '') as g:
  spamreader = csv.reader(g, quotechar='|')
  for row in spamreader:
    resi.append(row)
    

#convert bunch of rows to matrices
### ignore first 2 rowswhich correspond to 2 node matrix -- 1-regular graph
    
##########################
resi = resi[2:]
print(len(resi)/16)
n = int(len(resi)/16)
mat_list = []
for i in range(n):
  array = []
  for j in range(16):
    row = resi[i*16 +j]
    row = [float(k) for k in row]
    array.append(row)
  mat_list.append(array)

41.0


In [15]:
new_mat_list = mat_list[first_rem_index-1:35] #-1 for 2 node graph we deleted

In [16]:
def mats_to_graphs(mat_list):
  """
  Given list of adjaceny matrix repr of graphs, converts to actual graphs
  """
  g_list = []
  for mat in mat_list:
    arr = np.array(mat)
    G = nx.convert_matrix.from_numpy_matrix(arr)
    g_list.append(G)
  return g_list

In [17]:
g_list = mats_to_graphs(new_mat_list)

In [18]:
len(g_list)

9

# Execution Code

In [19]:
results = []
i =0
for g in g_list:
    i+=1
    print('###################################')
    print(i)
    result = opt_runs(g)
    results.append(result)

###################################
1
seed #0



tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4
tw 4



seed #1



seed #2



seed #3


...


seed #15



seed #16



seed #17



seed #18



seed #19


In [20]:
len(results)

9

In [21]:
len(g_list)

9

In [22]:
results

[array([ 0.05287721,  0.9054181 , 36.75809529, 33.33456477]),
 array([ 0.44709259,  0.09532931, 36.80660653, 35.29679668]),
 array([ 0.94691479,  0.09573749, 36.83566677, 31.61400921]),
 array([ 0.44688967,  0.09611521, 36.86069745, 35.80006044]),
 array([ 0.94714999,  0.09494843, 36.78111261, 32.17520016]),
 array([ 0.95166594,  0.58800554, 40.56774023, 39.73176679]),
 array([ 0.45203811,  0.41286263, 40.50479688, 39.77117587]),
 array([ 0.54821008,  0.58771491, 40.54663701, 26.71888972]),
 array([ 0.54833609,  0.5885663 , 40.60390962, 27.16771599])]

In [23]:
import scipy

def graphs_to_matrix(g_list):
  """
  Converts graphs to matrices
  """
  mat_list = []
  for g in g_list:
    mat = nx.linalg.graphmatrix.adj_matrix(g, weight = None)
    mat = scipy.sparse.csr_matrix.toarray(mat)
    mat_list.append(np.copy(mat))
  return mat_list


In [24]:
mat_list = graphs_to_matrix(g_list)

In [25]:
mat_list

[array([[0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1],
        [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
        [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0],
        [0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1],
        [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0],
        [1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1],
        [0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0],
        [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
        [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1],
        [0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]], dtype=int64),
 array([[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1],

# Storing Results

In [26]:
with open('16NodeRegularData_firsthalf_1_10_Jul2.csv', 'w', newline = '') as f:
  thewriter = csv.writer(f)
  for row in results :
    thewriter.writerow(row)

#Not Required
    
    # with open('16NodeRegularGraph_1_10_Jul1.csv', 'w', newline = '') as g:
#   thewriter = csv.writer(g)
#   for row in mat_list:
#     for new_row in row:
#       thewriter.writerow(new_row)